In [1]:
#import required packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
df = pd.read_csv("../data/external/potential-talents.csv").set_index('id')
df.head()

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [3]:
# drop fit feature as it contains no data
df.drop('fit', axis=1, inplace = True)
df.head()

,job_title,location,connection
id,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
2,Native English Teacher at EPIK (English Progra...,Kanada,500+
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
4,People Development Coordinator at Ryan,"Denton, Texas",500+
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(53, 3)

In [5]:
df.isnull().value_counts()

job_title  location  connection
False      False     False         53
dtype: int64

In [6]:
df.job_title.value_counts()

Aspiring Human Resources Professional                                                                                    2
2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional                 1
Lead Official at Western Illinois University                                                                             1
Senior Human Resources Business Partner at Heil Environmental                                                            1
Aspiring Human Resources Professional | An energetic and Team-Focused Leader                                             1
HR Manager at Endemol Shine North America                                                                                1
Human Resources professional for the world leader in GIS software                                                        1
RRP Brand Portfolio Executive at JTI (Japan Tobacco International)                                                       1
Information Syst

In [7]:
df.shape

(53, 3)

In [88]:
# print(df['job_title'].loc[0])
import re
import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def get_preprossed_text(text):
    pattern1 = r'[0-9]'
    pattern2 = r'[\n]'
    pattern3 = r'[^\s\w]'
    pattern4 = r'\s+'
    t1 = re.sub(pattern1, '', text)
    t2 = re.sub(pattern2, ' ', t1)
    t3 = re.sub(pattern3, '', t2)
    t4 = re.sub(pattern4, ' ', t3)
    t5 = t4.lower()
    lemmatizer = WordNetLemmatizer()
    tokenized_words =  word_tokenize(t5)
    lemmatized_words = [lemmatizer.lemmatize(tokens, pos='v') for tokens in tokenized_words] 
    processed_text = ' '.join(lemmatized_words)
    stop_words = stopwords.words('english')
    tokenized_text = word_tokenize(processed_text)
    stripped_words = []
    for word in tokenized_text:
        if word not in stop_words:
            stripped_words.append(word)
    processed_text = ' '.join(stripped_words)
    return processed_text

df['job_title'] = df.job_title.apply(get_preprossed_text)

In [89]:
df.replace({'job_title' : { 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources','hris' : 'human resources management system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'strategic and policy-making certification'
        , 'hr' : 'human resources'}}, regex=True, inplace=True)

In [90]:
from collections import Counter
words_counts = Counter()
for sentence in df.job_title:
    for word in sentence.split(' '):
        words_counts[word] += 1
print(len(words_counts))
print(words_counts.most_common())

182
[('human', 39), ('resources', 39), ('aspire', 13), ('professional', 10), ('seek', 10), ('manager', 7), ('university', 6), ('student', 6), ('management', 6), ('business', 5), ('generalist', 5), ('specialist', 4), ('position', 4), ('senior', 3), ('opportunities', 3), ('director', 3), ('major', 3), ('college', 2), ('graduate', 2), ('english', 2), ('coordinator', 2), ('officer', 2), ('internship', 2), ('retail', 2), ('staff', 2), ('north', 2), ('america', 2), ('information', 2), ('systems', 2), ('leader', 2), ('ct', 1), ('bauer', 1), ('magna', 1), ('cum', 1), ('laude', 1), ('native', 1), ('teacher', 1), ('epik', 1), ('program', 1), ('korea', 1), ('people', 1), ('development', 1), ('ryan', 1), ('advisory', 1), ('board', 1), ('member', 1), ('celal', 1), ('bayar', 1), ('humber', 1), ('system', 1), ('chapman', 1), ('vice', 1), ('president', 1), ('chief', 1), ('market', 1), ('communications', 1), ('corporate', 1), ('social', 1), ('responsibility', 1), ('engie', 1), ('houston', 1), ('woodlan

###### Bag of words

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

In [94]:

def rank_candidates_bag_of_words(keyword, df, feature_name, ngram):
    corpus = df[feature_name].tolist()
    keyword = get_preprossed_text(keyword)
    vectorizer = CountVectorizer( binary = True, ngram_range=(ngram))
    vectors = vectorizer.fit_transform(corpus)
    # corpus.append(keyword)
    print(vectors.toarray())
rank_candidates_bag_of_words('human resources', df, 'job_title', (2,2))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
